In [1]:
# !pip install -r requirements.txt

In [2]:
import os
import pdfx
import pandas as pd
# from tqdm import tqdm
from tqdm.notebook import tqdm

In [3]:
# get all the pdf files in the data directory
directories = ['05/','06/','07/','data/']
files = [[d+file for file in os.listdir(d)] for d in directories]
all_files = []
for f in files:
    all_files.extend(f)
files = all_files

In [4]:
# list to store the required data
data = []

In [5]:
following_string = 'Bitte überprüfen Sie den Kontoauszug. Ohne Ihren Gegenbericht innert 30 Tagen gilt er als genehmigt.'

In [6]:
remaining = []

In [7]:
# iterate over all the files
for file in tqdm(files):
    if file.endswith(".pdf"): # check if the file is a pdf file
        found = False
        pdf = pdfx.PDFx(file).get_text() # get the text from the pdf file
        # print(len(pdf))
        start = 0 # start index of the data
        while start<len(pdf)-5: # iterate until the end of the pdf file
            gutschrift, lastschrift, datum, text_ = '', '', '', ''
            index = pdf[start:].find("Total") # find the index of the word "Total"
            # print(index)
            if index == -1: # if the word "Total" is not found
                break # break the loop
            start += (index+5) # move the start index to the character after "Total"
            text = [t.strip() for t in pdf[start:start+150].strip().replace('\n\n','\n').split('\n')] # get the text from the pdf file, remove spaces, replace multiple new lines with single new line and split the lines into a list

            gutschrift = text[0] # get the gutschrift which is the first line of the text
            if gutschrift.count('.')>1: # if the gutschrift has more than one dot, then it is not a number
                if gutschrift.count('.')==2: # if the gutschrift has two dots, then it is a date
                    datum = gutschrift # the date is the gutschrift
                    text_ = text[1] # the text is the second line of the text

                    idx = pdf[:start].find(following_string) # find the index of the following string
                    if idx == -1: # if the following string is not found
                        break # break the loop

                    text1 = [t.strip() for t in pdf[idx-100:idx].strip().replace('\n\n','\n').split('\n')] # get the text from the pdf file, remove spaces, replace multiple new lines with single new line and split the lines into a list

                    gutschrift = text1[-3] # get the gutschrift which is the third line from the ending of the text
                    try:
                        gutschrift = float(gutschrift.replace(" ", "")) # convert the gutschrift to a float
                    except:
                        break # break the loop since the gutschrift should be a number

                    lastschrift = text1[-2] # get the lastschrift which is the second line from the ending of the text
                    if lastschrift.count('.')>1: # if the lastschrift has more than one dot, then it is not a number
                        break # break the loop since lastschrift should be a number
                    try:
                        lastschrift = float(lastschrift.replace(" ", "")) # convert the lastschrift to a float
                    except:
                        break # break the loop since lastschrift should be a number
                else:
                    break # gutschrift is not a number and not a date i.e, invalid data so break the loop
            else:
                try:
                    gutschrift = float(gutschrift.replace(" ", "")) # convert the gutschrift to a float
                except:
                    break # break the loop since the gutschrift should be a number

                lastschrift = text[1] # get the lastschrift which is the second line of the text

                if lastschrift.count('.')>1: # if the lastschrift has more than one dot, then it is not a number
                    break # break the loop since lastschrift should be a number
                try:
                    lastschrift = float(lastschrift.replace(" ", "")) # convert the lastschrift to a float
                except:
                    break # break the loop since the lastschrift should be a number

                if not datum:
                    datum = text[2] # get the datum which is the third line of the text
                if datum.count('.')!=2: # if the datum has more than two dots, then it is not a date
                    break # break the loop since datum should be a date

                if not text_:
                    text_ = text[3] # get the text which is the fourth line of the text

            data.append([datum, text_, gutschrift, lastschrift]) # append the data to the list
            found = True
            break # break the loop as every pdf file has only one desired data
        
        if not found:
            remaining.append(file)

  0%|          | 0/68 [00:00<?, ?it/s]

In [8]:
remaining # failed pdf will appear here

['data/21.pdf']

In [9]:
df_final = pd.DataFrame(data, columns=["Datum", "Text", "Gutschrift", "Lastschrift"])
df_final.to_csv("totals.csv", index=False)

In [10]:
df_final

,Datum,Text,Gutschrift,Lastschrift
0,02.05.22,Kontostand,41533.00,406.91
1,03.05.22,Kontostand,19216.55,144.68
2,04.05.22,Kontostand,22538.20,228384.21
3,05.05.22,Kontostand,24260.55,437.65
4,06.05.22,Kontostand,20938.90,419.16
...,...,...,...,...
62,15.07.22,Kontostand,18002.15,409.42
63,18.07.22,Kontostand,51165.20,384.99
64,19.07.22,Kontostand,13372.30,506.62
65,20.07.22,Kontostand,21531.35,18.92
